Importing necessary packages

In [71]:
import numpy as np
import pandas as pd
from mpl_toolkits.basemap import Basemap
import math
import geopy.distance

Calculating spacing to create tiles in QGIS

In [72]:
# Boundaries of SF: 37.708251, 37.811262, -122.514446, -122.357110

# vertical distance
coords_1 = (37.811262, -122.477010)
coords_2 = (37.708251, -122.477010)

vertical_dist = geopy.distance.geodesic(coords_1, coords_2).m
v_count = math.ceil(vertical_dist / 100) # we want each tiles to be about 100 m x 100 m

# horizontal distance 
coords_1 = (37.778946, -122.514446)
coords_2 = (37.778946, -122.357110)

horizontal_dist = geopy.distance.geodesic(coords_1, coords_2).m
h_count = math.ceil(horizontal_dist / 100) # we want each tiles to be about 100 m x 100 m

#  calculating intervals

long_interval = np.linspace(-122.357110, -122.514446, h_count)
lat_interval = np.linspace(37.708251, 37.811262, v_count)

print(long_interval[0] - long_interval[1])
print(lat_interval[0] - lat_interval[1])

0.0011401159420358908
-0.0009036052631543612


Importing QGIS created tiles with coordinates

In [130]:
tiles = pd.read_csv(r'.\Data\QGIS_Grid.csv')

Gathering list of tiles w/ road vector data

In [131]:
road_tiles = pd.read_csv(r'.\Data\QGIS_Intersection.csv')

In [132]:
road_tiles = pd.DataFrame(road_tiles['id'].unique())

Filter out unnecessary tiles

In [133]:
# Dropping tiles without road data

tiles = pd.concat([tiles.set_index('id'), road_tiles.set_index(0)], axis=1, join="inner").reset_index()

tiles = tiles.rename(columns={"index": "id"})

tiles = tiles.drop(columns=['wkt_geom'])

In [134]:
# Function to create midpoint
# https://code.activestate.com/recipes/577713-midpoint-of-two-gps-points/ 

def midpoint(x1, y1, x2, y2):
#Input values as degrees

#Convert to radians
    lat1 = math.radians(x1)
    lon1 = math.radians(x2)
    lat2 = math.radians(y1)
    lon2 = math.radians(y2)


    bx = math.cos(lat2) * math.cos(lon2 - lon1)
    by = math.cos(lat2) * math.sin(lon2 - lon1)
    lat3 = math.atan2(math.sin(lat1) + math.sin(lat2), \
           math.sqrt((math.cos(lat1) + bx) * (math.cos(lat1) \
           + bx) + by**2))
    lon3 = lon1 + math.atan2(by, math.cos(lat1) + bx)

    return [round(math.degrees(lat3), 6), round(math.degrees(lon3), 6)]

In [135]:
# Creating midpoints for each tile 
# Excluding tile if beyond SF boundaries on midpoint 

to_keep = pd.DataFrame(columns = ['id', 'Mid_lat', 'Mid_long'])

#bm = Basemap(projection='cyl', resolution='f') 

for i in range(len(tiles)):
    t = tiles.iloc[i]
    lat1 = t['top']
    lat2 = t['bottom']
    long1 = t['left']
    long2 = t['right']
    id = t['id']
    mid = midpoint(lat1, lat2, long1, long2)
    ypt,xpt = bm(mid[1], mid[0]) # converting to projection coordinates, enter parameters as long, lat 
    if mid[0] < 37.708251 or mid[0] > 37.811262 or mid[1] < -122.514446 or mid[1] > -122.357110: 
        continue # out of SF boundaries
    #elif bm.is_land(ypt,xpt) == False:
        #continue # not on land # shouldn't need because only include tiles with roads
    tk = pd.DataFrame([[id, mid[0], mid[1]]], columns = ['id', 'Mid_lat', 'Mid_long'])
    to_keep = pd.concat([to_keep, tk], ignore_index = True)

In [136]:
tiles = pd.concat([tiles.set_index('id'), to_keep.set_index('id')], axis=1, join="inner").reset_index()

C:\Users\arisa\Anaconda3\FIle\lib\site-packages\pandas\core\indexes\base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [137]:
tiles = tiles.rename(columns={"id": "Tile_ID", "left": "Long2", "right": "Long1", "top": "Lat2", "bottom": "Lat1"})

In [138]:
tiles['Tile_ID'] = tiles['Tile_ID'].astype('Int64')

Exporting to CSV

In [139]:
tiles.to_csv('.\Data\Tiles.csv', index=False)

In [140]:
tiles_2 = pd.read_csv(r'.\Data\Tiles.csv')

In [141]:
tiles_2.head(5)

,Tile_ID,Long2,Lat2,Long1,Lat1,Mid_lat,Mid_long
0,36,-122.514446,37.779636,-122.513306,37.778732,37.779184,-122.513876
1,37,-122.514446,37.778732,-122.513306,37.777829,37.778280,-122.513876
2,151,-122.513306,37.779636,-122.512166,37.778732,37.779184,-122.512736
3,152,-122.513306,37.778732,-122.512166,37.777829,37.778280,-122.512736
4,153,-122.513306,37.777829,-122.512166,37.776925,37.777377,-122.512736
